In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

# Define the file paths
file_path1 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_training_data.csv'
file_path2 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_testing_data.csv'
file_path3 = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_training_data.csv'
file_path4  = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_testing_data.csv'
# Use pandas to read the CSV files and then convert them to NumPy arrays
caltech_train = pd.read_csv(file_path1).values
caltech_test = pd.read_csv(file_path2).values

JPL_train = pd.read_csv(file_path3).values
JPL_test=pd.read_csv(file_path4).values

In [ ]:
JPL_test[:5]

In [ ]:
filtered_rows = JPL_test[JPL_test[:, 4] == 356]

# Printing the filtered rows
print(filtered_rows)

In [ ]:
import pandas as pd
import numpy as np

# Define the file paths
file_path1 = '/content/drive/MyDrive/SuperUROP /Data Analysis/caltech_training_data_full.csv'
file_path3 = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_training_data.csv'
file_path4 = '/content/drive/MyDrive/SuperUROP /Data Analysis/JPL_testing_data.csv'

# Use pandas to read the CSV files
caltech_full_df = pd.read_csv(file_path1)

JPL_train_df = pd.read_csv(file_path3)
JPL_test_df = pd.read_csv(file_path4)

# Filter the caltech_full DataFrame
caltech_full_df = caltech_full_df[caltech_full_df['duration'] <= 24]

caltech_train_df = caltech_full_df[(caltech_full_df['connectionTime'] >= '2021-03-01') & (caltech_full_df['connectionTime'] <= '2021-05-31')]
caltech_train_df= caltech_train_df[caltech_train_df['no_sessions'] >= 50]
caltech_test_df = caltech_full_df[(caltech_full_df['connectionTime'] >= '2021-06-01') & (caltech_full_df['connectionTime'] <= '2021-06-30')]

JPL_train_df=JPL_train_df[JPL_train_df['no_sessions'] >= 30]

# Convert to NumPy arrays if necessary
caltech_train = caltech_train_df.values
caltech_test = caltech_test_df.values
JPL_train = JPL_train_df.values
JPL_test = JPL_test_df.values


#Data Processing for mode

In [ ]:
JPL_train=JPL_train[:,1:]
JPL_test=JPL_test[:,1:]

sorted_JPL_training = sorted(JPL_train, key=lambda x: x[3])
sorted_JPL_training=np.delete(sorted_JPL_training, [1], axis=1)
sorted_JPL_testing = sorted(JPL_test, key=lambda x: x[3])
sorted_JPL_testing=np.delete(sorted_JPL_testing, [1], axis=1)

def convert_to_interval(datetime_str):
    # Extract time string from the datetime string
    time_str = datetime_str.split()[1]

    # Convert time string into hours, minutes, and seconds
    hours, minutes, seconds = map(int, time_str.split(':'))

    # Calculate total minutes
    total_minutes = hours * 60 + minutes

    # Find the nearest lower multiple of 15 for the minutes
    interval_minutes = (total_minutes // 15) * 15

    # Convert back to hour and minute
    interval_hour, interval_minute = divmod(interval_minutes, 60)

    # Return as a formatted string
    return "{:02d}:{:02d}".format(interval_hour, interval_minute)

sorted_JPL_training[:, 0] = np.vectorize(convert_to_interval)(sorted_JPL_training[:, 0])
sorted_JPL_testing[:, 0] = np.vectorize(convert_to_interval)(sorted_JPL_testing[:, 0])

In [ ]:
sorted_JPL_training[:, 3] = sorted_JPL_training[:, 3].astype(float) * 60
sorted_JPL_training[:, 3] = np.round(sorted_JPL_training[:, 3].astype(float) / 10) * 10
sorted_JPL_testing[:, 3] = sorted_JPL_testing[:, 3].astype(float) * 60
sorted_JPL_testing[:, 3] = np.round(sorted_JPL_testing[:, 3].astype(float) / 10) * 10

In [ ]:
#Make training and testing set have the same user IDs
users_from_training = set(sorted_JPL_training[:, 2])
mask = np.isin(sorted_JPL_testing[:, 2], list(users_from_training))
sorted_JPL_testing = sorted_JPL_testing[mask]
users_from_testing = set(sorted_JPL_testing[:, 2])
mask = np.isin(sorted_JPL_training[:, 2], list(users_from_testing))
sorted_JPL_training = sorted_JPL_training[mask]

In [ ]:
# Convert time strings to total minutes for sorting
total_minutes_JPL_training = np.array([int(time.split(':')[0])*60 + int(time.split(':')[1]) for time in sorted_JPL_training[:, 0]])
total_minutes_JPL_testing = np.array([int(time.split(':')[0])*60 + int(time.split(':')[1]) for time in sorted_JPL_testing[:, 0]])
# Argsort first by the 3rd column and then by total_minutes
indices_JPL_training = np.lexsort((total_minutes_JPL_training, sorted_JPL_training[:, 2].astype(int)))
indices_JPL_testing = np.lexsort((total_minutes_JPL_testing, sorted_JPL_testing[:, 2].astype(int)))
# Use the sorted indices to reorder the array
sorted_data_JPL_training = sorted_JPL_training[indices_JPL_training]
sorted_data_JPL_testing = sorted_JPL_testing[indices_JPL_testing]

In [ ]:
sorted_data_JPL_testing

#Data Processing for MLR & DKDE

In [ ]:
#Remove departure time (2nd column)
# Convert arrival date to hour and find day of the week
from datetime import datetime

def convert_time_and_day(data_array):
    """
    Converts the time from HH:MM to HH.XX format and appends the day of the week to it.
    Also, removes the second column.
    """
    transformed_data = []
    for row in data_array:
        # Convert the arrival time to HH.XX format
        time_obj = datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S')
        new_time = time_obj.hour + (time_obj.minute / 60.0)

        # Convert the date to a day of the week
        day_of_week = time_obj.strftime('%A')
        new_time = str(new_time) + " " + day_of_week

        # Create a new row excluding the second column
        new_row = [new_time] + list(row[2:])
        transformed_data.append(new_row)

    return np.array(transformed_data)

JPL_train=convert_time_and_day(JPL_train)
JPL_test=convert_time_and_day(JPL_test)

In [ ]:
def day_to_number(day):
    """Converts a day of the week to its corresponding discrete value."""
    days = {
        'Monday': 1,
        'Tuesday': 2,
        'Wednesday': 3,
        'Thursday': 4,
        'Friday': 5,
        'Saturday': 6,
        'Sunday': 7
    }
    return days[day]

def separate_time_and_day(data_array):
    """
    Separates the time and day in the given column,
    and converts the day into a discrete value between 1 and 7.

    """
    transformed_data = []
    for row in data_array:
        time_day_str = row[0]
        time, day = time_day_str.split()
        time = float(time)
        day_num = day_to_number(day)

        # Create a new row with separated time and day number
        new_row = [time, day_num] + list(row[1:])
        transformed_data.append(new_row)

    return np.array(transformed_data)

JPL_train=separate_time_and_day(JPL_train)
JPL_test=separate_time_and_day(JPL_test)

In [ ]:
#Make training and testing set have the same user IDs
users_from_training_caltech = set(caltech_train[:, 3])
mask_caltech = np.isin(caltech_test[:, 3], list(users_from_training_caltech))
caltech_test = caltech_test[mask_caltech]
users_from_testing_caltech = set(caltech_test[:, 3])
mask_caltech = np.isin(caltech_train[:, 3], list(users_from_testing_caltech))
caltech_train = caltech_train[mask_caltech]

users_from_training = set(JPL_train[:, 3])
mask = np.isin(JPL_test[:, 3], list(users_from_training))
JPL_test = JPL_test[mask]
users_from_testing = set(JPL_test[:, 3])
mask = np.isin(JPL_train[:, 3], list(users_from_testing))
JPL_train = JPL_train[mask]

In [ ]:
JPL_train = np.array(JPL_train, dtype='float')
JPL_test = np.array(JPL_test, dtype='float')

#Data Processing for correlation

In [ ]:
from collections import defaultdict

# Organize the arrival times by user ID for the entire dataset
user_times_JPL = defaultdict(list)


# Accumulate all arrival times for each user in JPL_train
for row in JPL_train:
    arrival_time = float(row[0])
    user_id = row[3]
    user_times_JPL[user_id].append(arrival_time)

# Define the functions for hourly and half-hourly representations
def create_hourly_representation(times):
    hourly_vector = [0] * 24
    for time in times:
        hour = int(float(time))
        hourly_vector[hour] += 1
    return hourly_vector

def create_half_hourly_representation(times):
    half_hourly_vector = [0] * 48
    for time in times:
        interval = int(float(time) * 2)  # Multiply by 2 for half-hourly intervals
        half_hourly_vector[interval] += 1
    return half_hourly_vector

# Compute the hourly and half-hourly representations for each user
hourly_representations_JPL = {}
half_hourly_representations_JPL = {}

for user_id, times in user_times_JPL.items():
    hourly_representations_JPL[user_id] = create_hourly_representation(times)
    half_hourly_representations_JPL[user_id] = create_half_hourly_representation(times)



In [ ]:
import numpy as np
from collections import defaultdict

# Organize the energy consumption durations by user ID for the entire dataset
user_durations_JPL = defaultdict(list)


# Accumulate all energy consumption durations for each user in JPL_train
for row in JPL_train:
    stay_duration = float(row[4])
    user_id = row[3]
    user_durations_JPL[user_id].append(stay_duration)

# Define functions for hourly and half-hourly representations
def create_hourly_representation_duration(durations):
    hourly_vector = [0] * 24
    for duration in durations:
        index = int(float(duration))
        hourly_vector[index] += 1
    return hourly_vector

def create_half_hourly_representation_duration(durations):
    half_hourly_vector = [0] * 48
    for duration in durations:
        interval = int(float(duration) * 2)
        half_hourly_vector[interval] += 1
    return half_hourly_vector

# Compute the hourly and half-hourly representations for each user
hourly_representations_duration_JPL = {}
half_hourly_representations_duration_JPL = {}

for user_id, durations in user_durations_JPL.items():
    hourly_representations_duration_JPL[user_id] = create_hourly_representation_duration(durations)
    half_hourly_representations_duration_JPL[user_id] = create_half_hourly_representation_duration(durations)


In [ ]:
def concatenate_representations(dict1, dict2):
    concatenated = {}
    for key in set(dict1.keys()) | set(dict2.keys()):  # union of keys from both dicts
        list1 = dict1.get(key, [])
        list2 = dict2.get(key, [])

        # Concatenate the lists
        concatenated_list = list1 + list2

        concatenated[key] = concatenated_list

    return concatenated

# Example usage

concatenated_hourly_JPL = concatenate_representations(hourly_representations_JPL, hourly_representations_duration_JPL)
concatenated_half_hourly_JPL = concatenate_representations(half_hourly_representations_JPL, half_hourly_representations_duration_JPL)



#Calculate correlations - choose one of the two
Run one of the two cells as the results have the same name

##Calculate cosine similarity

In [ ]:
from sklearn.metrics import pairwise_distances
import pandas as pd

def data_to_dataframe(data):
    return pd.DataFrame.from_dict(data, orient='index')

def compute_cosine_similarity(df):
    cosine_distance = pairwise_distances(df, metric='cosine')
    cosine_similarity = 1 - cosine_distance

    # Set the user IDs as row and column names to preserve them
    cosine_similarity_df = pd.DataFrame(cosine_similarity, index=df.index, columns=df.index)
    return cosine_similarity_df

# Compute cosine similarity for each concatenated dataset
df_hourly_JPL = data_to_dataframe(concatenated_hourly_JPL)
similarities_hourly_JPL = compute_cosine_similarity(df_hourly_JPL)

df_half_hourly_JPL = data_to_dataframe(concatenated_half_hourly_JPL)
similarities_half_hourly_JPL = compute_cosine_similarity(df_half_hourly_JPL)



##Calculate Pearson correlation

In [ ]:
import numpy as np

def compute_pearson_correlation(v1, v2):
    # Handling cases where vectors are too short or contain constant values
    if len(v1) > 1 and len(v2) > 1 and np.std(v1) * np.std(v2) != 0:
        return np.corrcoef(v1, v2)[0, 1]
    else:
        return None  # Returning None for cases where correlation is not defined

import pandas as pd

def compute_pearson_correlation_matrix_df(representations):
    user_ids = list(representations.keys())
    n = len(user_ids)
    correlation_matrix = np.zeros((n, n))

    for i in range(n):
        for j in range(i, n):  # Compute only for one triangle and mirror it as the matrix is symmetric
            if i != j:
                corr = compute_pearson_correlation(representations[user_ids[i]], representations[user_ids[j]])
                correlation_matrix[i, j] = corr if corr is not None else 0
                correlation_matrix[j, i] = correlation_matrix[i, j]  # Mirror the value
            else:
                correlation_matrix[i, j] = 1  # Self-correlation is always 1

    # Convert to DataFrame for better usability
    correlation_df = pd.DataFrame(correlation_matrix, index=user_ids, columns=user_ids)
    return correlation_df

similarities_hourly_JPL = compute_pearson_correlation_matrix_df(concatenated_hourly_JPL)
similarities_half_hourly_JPL = compute_pearson_correlation_matrix_df(concatenated_half_hourly_JPL)




In [ ]:
from scipy.stats import spearmanr
import pandas as pd

def data_to_dataframe(data):
    return pd.DataFrame.from_dict(data, orient='index')

def calculate_spearman_correlation(df):
    # Transpose the DataFrame to make users (rows) into columns
    df_transposed = df.T

    # Initialize an empty DataFrame to store the results
    spearman_corr_df = pd.DataFrame(index=df.index, columns=df.index)

    # Calculate Spearman correlation for each pair of users (now columns)
    for user1 in df_transposed.columns:
        for user2 in df_transposed.columns:
            # Calculate Spearman correlation
            corr, _ = spearmanr(df_transposed[user1], df_transposed[user2])
            spearman_corr_df.loc[user1, user2] = corr

    # Convert all values to numeric
    return spearman_corr_df.apply(pd.to_numeric)

# Compute Spearman correlation for each concatenated dataset
df_hourly_JPL = data_to_dataframe(concatenated_hourly_JPL)
similarities_hourly_JPL = calculate_spearman_correlation(df_hourly_JPL)

df_half_hourly_JPL = data_to_dataframe(concatenated_half_hourly_JPL)
similarities_half_hourly_JPL = calculate_spearman_correlation(df_half_hourly_JPL)


#Calculated most correlated users (using raw values)
Run one of the two cells as the results have the same name - raw values perform much better

In [ ]:
#Using threshold
def most_correlated_users(matrix,cap,threshold=0.65):
    """
    For each user, find the top correlated users with correlations above 0.5.

    :param matrix: Cosine similarity matrix.
    :param threshold: Minimum correlation threshold for considering a user as correlated.
    :return: Dictionary with keys being user IDs and values being lists of top correlated user IDs.
    """
    result = {}

    for user_index in matrix.index:
        # Subtract self-similarity value for the current user
        user_similarities = matrix.loc[user_index].drop(user_index)
        # Filter users with correlation above the threshold
        filtered_users = user_similarities[abs(user_similarities) > threshold]

        # Sort by descending correlation
        sorted_users = filtered_users.sort_values(ascending=False)

        # Append users above the threshold to the result
        result[user_index] = sorted_users.index.tolist()

    return result

# Assuming `similarities_hourly_cal` is your cosine similarity matrix for the entire dataset

# Similarly for JPL data
cap=5
most_corr_users_hourly_JPL = most_correlated_users(similarities_hourly_JPL,cap)
most_corr_users_half_hourly_JPL = most_correlated_users(similarities_half_hourly_JPL,cap)

In [ ]:
similarities_hourly_JPL

In [ ]:
most_corr_users_hourly_JPL

#Mode - Stay duration prediction (no correlation)

In [ ]:
#Mode calculation with linear interpolation
from collections import Counter
def compute_mode_per_user_per_interval(data):
    """Compute the mode per user per time interval with linear interpolation."""
    user_groups = {}
    mode_per_user_per_interval = {}

    # Group by user
    for row in data:
        if row[2] not in user_groups:
            user_groups[row[2]] = []
        user_groups[row[2]].append(row)

    # Calculate modes per user
    for user, entries in user_groups.items():
        time_intervals = sorted(list(set([entry[0] for entry in entries])))
        modes = {}
        for interval in time_intervals:
            session_values = [entry[3] for entry in entries if entry[0] == interval]
            counts = Counter(session_values)
            highest_freq = max(counts.values())
            common_vals = [key for key, val in counts.items() if val == highest_freq]

            # If there's a single mode, use it; else, average them
            modes[interval] = sum(common_vals) / len(common_vals)

        # Linearly interpolate for missing modes
        all_intervals = sorted(list(set(data[:, 0])))
        for i, interval in enumerate(all_intervals):
            if interval not in modes:
                # Find previous and next known modes
                prev_mode = next((modes[prev_int] for prev_int in reversed(all_intervals[:i]) if prev_int in modes), None)
                next_mode = next((modes[next_int] for next_int in all_intervals[i+1:] if next_int in modes), None)

                # If both previous and next modes exist, interpolate
                if prev_mode is not None and next_mode is not None:
                    gap_size = all_intervals[i+1:].index(next((next_int for next_int in all_intervals[i+1:] if next_int in modes))) + 1
                    increment = (next_mode - prev_mode) / (gap_size + 1)
                    modes[interval] = prev_mode + increment

                # If no next mode exists, keep the mode same as the previous mode
                elif prev_mode is not None:
                    modes[interval] = prev_mode

                # If no previous mode exists, keep the mode same as the next mode
                elif next_mode is not None:
                    modes[interval] = next_mode

        mode_per_user_per_interval[user] = modes

    return mode_per_user_per_interval

# Update mode_per_user_per_interval using the new function
mode_per_user_per_interval_JPL_interpolate = compute_mode_per_user_per_interval(sorted_data_JPL_training)

#MLR Stay Duration Prediction for all users (no correlation)

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Sample training data
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def mlr_model(train_data, test_data, user_id):
    # Filter training and testing data for the specific user
    user_train_data = train_data[train_data[:, 3] == user_id]
    user_test_data = test_data[test_data[:, 3] == user_id]

    # Independent variables are arrival time, day of the week
    X_train = user_train_data[:, :2]
    X_test = user_test_data[:, :2]

    # Dependent variable is the energy
    y_train = user_train_data[:, 4]
    y_test = user_test_data[:, 4]

    # Train the model
    model = LinearRegression().fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Calculate user SMAPE
    n = len(y_test)
    smape_val = (1/ n) * np.sum(np.abs(y_test - y_pred) / (np.abs(y_test+y_pred)))*100

    return model





In [ ]:
MLR_user_models={}
user_ids_JPL = np.unique(np.concatenate((JPL_train[:, 3], JPL_test[:, 3])))

# Loop through each user ID in the JPL dataset
for user_id in user_ids_JPL:
    model = mlr_model(JPL_train, JPL_test, user_id)
    MLR_user_models[user_id]=model


#Train DKDE models for all users

In [ ]:
! pip install KDE-diffusion

In [ ]:
import numpy as np
from kde_diffusion import kde2d
user_ids_JPL = np.unique(np.concatenate((JPL_train[:, 3], JPL_test[:, 3])))

class KDEModel:
    def __init__(self, grid, density):
        self.grid = grid
        self.density = density

    def predict(self, X):
        predictions = [self._predict_y_given_x(x) for x in X]
        return predictions

    def _predict_y_given_x(self, new_x):
        # Find the closest x index
        x_idx = np.argmin(np.abs(self.grid[0] - new_x))

        # Get the y values and their corresponding densities for the given x
        y_values = self.grid[1]
        y_densities = self.density[x_idx]

        # Find the y with the maximum density
        predicted_y = y_values[np.argmax(y_densities)]

        return predicted_y

def dkde_model(train_data, user_id):
    # Filter training data for the specific user
    user_train_data = train_data[train_data[:, 3] == user_id]

    # Extract columns for stay duration and energy consumption
    X_train = user_train_data[:, 0]  # Stay duration
    Y_train = user_train_data[:, 4]  # Energy consumption

    # Perform KDE
    density, grid, bandwidth = kde2d(X_train, Y_train, n=64, limits=None)

    # Create and return a KDE model instance
    kde_model = KDEModel(grid, density)
    return kde_model

DKDE_user_models = {user_id: dkde_model(JPL_train, user_id) for user_id in user_ids_JPL}


#Train Decision Tree model for all users

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler

def train_user_model(train_data, user_id,depth_range=(1, 21), min_samples_split_range=(2, 11), cv_folds=5):
    user_train_data = train_data[train_data[:, 3] == user_id]
    X_train = user_train_data[:, 0].reshape(-1, 1)  # Arrival time
    y_train = user_train_data[:, 4]  # Stay duration

    # Set up grid search with cross-validation for Decision Tree
    param_grid = {
        'max_depth': list(range(depth_range[0], depth_range[1])),
        'min_samples_split': list(range(min_samples_split_range[0], min_samples_split_range[1]))
    }
    dt = DecisionTreeRegressor()
    grid_search = GridSearchCV(dt, param_grid, cv=cv_folds, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)

    # Train the model using the optimal k
    model = grid_search.best_estimator_


    return model

DT_user_models = {user_id: train_user_model(JPL_train, user_id) for user_id in user_ids_JPL}

#Train SVR models for all users

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
def train_user_model(train_data, user_id, k=5):
    user_train_data = train_data[train_data[:, 3] == user_id]
    X_train = user_train_data[:, 0].reshape(-1, 1)  # Arrival time
    y_train = user_train_data[:, 4]  # Stay duration

   # Define the hyperparameters to be optimized
    param_grid = {
        'C': [0.1, 1, 10, 100],
        'epsilon': [0.001, 0.01, 0.1, 1],
        'gamma': ['scale', 'auto', 0.1, 1, 10]
    }

    # Initialize SVR with RBF kernel
    svr_rbf = SVR(kernel='rbf')

    # Initialize GridSearchCV with k-fold cross-validation
    grid_search = GridSearchCV(estimator=svr_rbf, param_grid=param_grid, cv=k, scoring='neg_mean_squared_error', n_jobs=-1)

    # Fit the model to the training data
    grid_search.fit(X_train, y_train)

    model = grid_search.best_estimator_

    return model
svr_models_JPL = {user_id: train_user_model(JPL_train, user_id) for user_id in user_ids_JPL}

#Enseble ML Algorithm Stay Duration (with correlations)

In [ ]:
R_SD={'169': 2.2551554266668488,
 '171': 3.7531759242694362,
 '176': 3.033506931058353,
 '220': 4.157056240805807,
 '322': 2.929926450352931,
 '334': 1.5403402896510696,
 '335': 4.182440356147159,
 '346': 4.63088280381524,
 '365': 4.612058546331858,
 '368': 3.8637314436567745,
 '372': 3.4779871035895753,
 '374': 3.7627475450103334,
 '378': 3.61759981491824,
 '382': 4.358454020839778,
 '404': 4.285879042160351,
 '405': 4.760574930778141,
 '406': 1.1510712960433436,
 '409': 3.3539069505133905,
 '410': 4.301005502712871,
 '416': 4.199233283013298,
 '436': 4.091891479434165,
 '444': 3.529125348909349,
 '458': 4.060001379997106,
 '467': 4.019928954004856,
 '474': 3.2450469879409285,
 '476': 4.393288833959378,
 '481': 4.429827240449028,
 '483': 3.079068807362642,
 '507': 4.336149365824373,
 '526': 2.4205249888839466,
 '531': 3.9517493146670986,
 '537': 3.1479021761196098,
 '551': 4.881572582872202,
 '553': 3.0305502893037675,
 '576': 3.628599820757222,
 '577': 4.319531561288392,
 '581': 3.4796093044547454,
 '592': 3.1009000797824084,
 '607': 4.831651402991986,
 '651': 3.6013651820057837,
 '726': 3.7379880870161575,
 '742': 4.150297519178871,
 '826': 3.710092899316145,
 '933': 4.41718644718856}

R_SD = {int(key): value for key, value in R_SD.items()}

In [ ]:
def ensemble_algorithm_with_correlation(train_data, test_data,test_data_mode, user_id,most_corr_users, mlr_user_model, DT_user_model,similarity_matrix, R_SD):
    # Filter training and testing data for the specific user
    user_test_data = test_data[test_data[:, 3] == user_id]
    user_test_data_mode = test_data_mode[test_data_mode[:, 2] == user_id]

    X_test_mlr = user_test_data[:, :2]
    y_test = user_test_data[:, 4]
    #y_test=user_test_data_mode[:, 3]
    X_test_dt = user_test_data[:, 0].reshape(-1, 1)

    # Choose and use the model based on the R_SD ratio for the main user
    if R_SD.get(user_id, 0) < 4.5:
        user_model =mlr_user_model[user_id]
        y_pred = user_model.predict(X_test_mlr)
    else:
        user_model = DT_user_model[user_id]
        y_pred = user_model.predict(X_test_dt)

    # Convert y_pred to a NumPy array if it is a list
    y_pred = np.array(y_pred) if isinstance(y_pred, list) else y_pred

    # Adjust the prediction based on the most correlated users
    total_correlation = 1
    if user_id in most_corr_users:
        correlated_users = most_corr_users[user_id]

        for other_user_id in correlated_users:
            if other_user_id in mlr_user_model or other_user_id in DT_user_model:
                correlation = similarity_matrix.loc[user_id, other_user_id]

                # Choose the model for the correlated user based on R_SD
                if R_SD.get(other_user_id, 0) < 4.5:
                    other_user_model = mlr_user_model[other_user_id]
                    other_user_pred = other_user_model.predict(X_test_mlr)
                else:
                    other_user_model = DT_user_model[other_user_id]
                    other_user_pred = other_user_model.predict(X_test_dt)

                # Convert other_user_pred to a NumPy array if it's a list
                other_user_pred = np.array(other_user_pred) if isinstance(other_user_pred, list) else other_user_pred

                # Ensure y_pred is an array before the operation
                if not isinstance(y_pred, np.ndarray):
                    y_pred = np.array(y_pred)

                y_pred += other_user_pred * correlation
                total_correlation += correlation

    # Ensure y_pred is an array before the division
    if not isinstance(y_pred, np.ndarray):
        y_pred = np.array(y_pred)

    if total_correlation != 0:
        y_pred /= total_correlation

    # Calculate user SMAPE
    n = len(y_test)
    smape_val = (1 / n) * np.sum(np.abs(y_test - y_pred) / (np.abs(y_test) + np.abs(y_pred))) * 100

    return smape_val



In [ ]:
user_ids_JPL = np.unique(np.concatenate((JPL_train[:, 3], JPL_test[:, 3])))

smape_list_JPL=[]

for user_id in user_ids_JPL:
    smape = ensemble_algorithm_with_correlation(JPL_train, JPL_test,sorted_data_JPL_testing, user_id, most_corr_users_hourly_JPL, MLR_user_models,DT_user_models,similarities_hourly_JPL,R_SD)
    smape_list_JPL.append(smape)

# Calculate the average SMAPE
average_smape = np.mean(smape_list_JPL) if smape_list_JPL else 0

print(f"Average SMAPE for the dataset: {average_smape:.2f}%")

In [ ]:
import numpy as np

# Assuming you already have the variables and functions defined as per your code snippet
user_ids_JPL = np.unique(np.concatenate((JPL_train[:, 3], JPL_test[:, 3])))

# Initialize an empty dictionary to store userID as key and SMAPE as value
smape_dict_JPL = {}

for user_id in user_ids_JPL:
    smape = ensemble_algorithm_with_correlation(JPL_train, JPL_test,sorted_data_JPL_testing, user_id, most_corr_users_hourly_JPL, MLR_user_models,DT_user_models,similarities_hourly_JPL,R_SD)
    smape_dict_JPL[user_id] = smape  # Store the SMAPE in the dictionary with user_id as key

# Calculate the average SMAPE using the dictionary values
average_smape = np.mean(list(smape_dict_JPL.values())) if smape_dict_JPL else 0

print(f"Average SMAPE for the dataset: {average_smape:.2f}%")


In [ ]:
import numpy as np

# Assuming userIDs are at index 3 and no_sessions is at the correct index (let's assume it's at index -1 for this example)
user_ids = JPL_train[:, 3]
no_sessions = JPL_train[:, -1]  # Replace -1 with the actual index of no_sessions

# Filter userIDs where no_sessions is greater than 30
# Note: Ensure no_sessions is of type int if not, convert it using no_sessions.astype(int)
user_ids_more_than_30_sessions = user_ids[no_sessions.astype(int) >= 30]

# Since user_ids can have duplicates, get unique userIDs
unique_user_ids_more_than_30_sessions = np.unique(user_ids_more_than_30_sessions)

# Convert to list if needed
unique_user_ids_more_than_30_sessions_list = unique_user_ids_more_than_30_sessions.tolist()

# Now unique_user_ids_more_than_30_sessions_list contains the userIDs with more than 30 sessions
print(unique_user_ids_more_than_30_sessions_list)


In [ ]:
# Assuming smape_dict_JPL is the dictionary with userID as key and SMAPE as value
# And unique_user_ids_more_than_30_sessions_list contains the list of userIDs

# Initialize an empty list to store SMAPE values for users with more than 30 sessions
smape_values_for_selected_users = []

# Loop through each userID in the list and get their SMAPE from the dictionary
for user_id in unique_user_ids_more_than_30_sessions_list:
    if user_id in smape_dict_JPL:
        smape_values_for_selected_users.append(smape_dict_JPL[user_id])
    else:
        print(f"No SMAPE found for userID {user_id}")

# If you want to calculate the average SMAPE for these users
if smape_values_for_selected_users:
    average_smape_selected_users = np.mean(smape_values_for_selected_users)
    print(f"Average SMAPE for users with more than 30 sessions: {average_smape_selected_users:.2f}%")
else:
    print("No SMAPE values found for the selected users.")


#Ensemble machine learning with mode

In [ ]:
# for user_id, time_slots in mode_per_user_per_interval_JPL_interpolate.items():
#     for time_slot, mode_value in time_slots.items():
#         # Divide each mode value by 60 and update the dictionary
#         mode_per_user_per_interval_JPL_interpolate[user_id][time_slot] = mode_value


In [ ]:
import datetime
def generate_15min_intervals():
    intervals = []
    for hour in range(24):
        for minute in [0, 15, 30, 45]:
            intervals.append(f'{hour:02d}:{minute:02d}')
    return intervals

all_intervals = generate_15min_intervals()

def interpolate_modes(user_modes, all_intervals):
    existing_times = [datetime.datetime.strptime(time, '%H:%M') for time in user_modes.keys()]
    existing_values = list(user_modes.values())

    # Convert times to minutes since midnight for interpolation
    existing_minutes = [(time.hour * 60 + time.minute) for time in existing_times]
    all_minutes = [(datetime.datetime.strptime(time, '%H:%M').hour * 60 + datetime.datetime.strptime(time, '%H:%M').minute) for time in all_intervals]

    interpolated_values = np.interp(all_minutes, existing_minutes, existing_values)

    # Map interpolated values back to time slots
    return dict(zip(all_intervals, interpolated_values))

# Interpolate for each user
for user_id, user_modes in mode_per_user_per_interval_JPL_interpolate.items():
    mode_per_user_per_interval_JPL_interpolate[user_id] = interpolate_modes(user_modes, all_intervals)

In [ ]:
import numpy as np
import datetime

def interpolate_modes_full(user_modes, all_intervals):
    if not user_modes:  # Check if user_modes is empty
        return dict.fromkeys(all_intervals, 0)  # Return a dictionary with 0 for all intervals if no modes

    # Parse the times and modes from the user data
    existing_times = [datetime.datetime.strptime(time, '%H:%M') for time in user_modes.keys()]
    existing_values = list(user_modes.values())

    # Find the earliest and latest times for the user
    earliest_time = min(existing_times)
    latest_time = max(existing_times)

    # Initialize dictionaries to hold the full set of modes
    full_modes = {}

    # Repeat the earliest real value for times before the first observation
    for interval in all_intervals:
        interval_time = datetime.datetime.strptime(interval, '%H:%M')
        if interval_time < earliest_time:
            full_modes[interval] = existing_values[0]  # Use the first (earliest) mode value
        elif interval_time > latest_time:
            full_modes[interval] = existing_values[-1]  # Use the last (latest) mode value

    # Interpolate between the first and last observations
    existing_minutes = [(time.hour * 60 + time.minute) for time in existing_times]
    all_minutes = [(datetime.datetime.strptime(time, '%H:%M').hour * 60 + datetime.datetime.strptime(time, '%H:%M').minute) for time in all_intervals if datetime.datetime.strptime(time, '%H:%M') >= earliest_time and datetime.datetime.strptime(time, '%H:%M') <= latest_time]

    interpolated_values = np.interp(all_minutes, existing_minutes, existing_values)

    # Map interpolated values to their corresponding time slots
    for time, value in zip(all_intervals, interpolated_values):
        if datetime.datetime.strptime(time, '%H:%M') >= earliest_time and datetime.datetime.strptime(time, '%H:%M') <= latest_time:
            full_modes[time] = value

    return full_modes

# Interpolate for each user, assuming mode_per_user_per_interval_JPL_interpolate is defined
for user_id, user_modes in mode_per_user_per_interval_JPL_interpolate.items():
    mode_per_user_per_interval_JPL_interpolate[user_id] = interpolate_modes_full(user_modes, all_intervals)


In [ ]:
import pandas as pd
import numpy as np

# Assuming mode_per_user_per_interval_caltech_interpolate is your prediction dataset
# and correlation_matrix is your correlation matrix

def weighted_average_prediction(user_id, time_slot, mode_per_user_per_interval, most_corr_users, correlation_matrix):
    # Initialize weighted predictions with the user's own prediction or a default value
    user_prediction = mode_per_user_per_interval.get(user_id, {}).get(time_slot, 0)  # You can change 0 to a default value if needed
    weighted_predictions = user_prediction
    total_correlation=1
    # Add weighted predictions of the most correlated users
    for corr_user_id in most_corr_users[user_id]:
        corr_value = correlation_matrix.loc[user_id, corr_user_id]
        corr_user_prediction = mode_per_user_per_interval.get(corr_user_id, {}).get(time_slot, 0)  # You can change 0 to a default value if needed

        # Add to weighted predictions
        weighted_predictions += corr_user_prediction * corr_value
        total_correlation+=corr_value
    weighted_predictions=weighted_predictions/total_correlation
    return weighted_predictions

In [ ]:
from collections import defaultdict

# Initialize dictionaries to store true and predicted values for each user
y_true_by_user = defaultdict(list)
y_pred_by_user = defaultdict(list)

# Function to calculate SMAPE
def calculate_smape(y_true, y_pred):
    """Compute the SMAPE"""
    n = len(y_true)
    if n == 0:
        return np.nan  # Return NaN if there are no observations
    smape_val = (1/n) * np.sum(np.abs(y_pred - y_true) / (np.abs(y_true + y_pred))) * 100
    return smape_val

# Iterate over the testing data to fill the dictionaries
for row in sorted_data_JPL_testing:
    user_id = row[2]  # Assuming user_id is at index 2
    time_slot = row[0]  # Assuming time_slot is at index 0

    if time_slot in mode_per_user_per_interval_JPL_interpolate.get(user_id, {}):
        y_true_by_user[user_id].append(row[3])  # true_value is assumed to be at index 3

        # Calculate the weighted prediction
        weighted_pred = weighted_average_prediction(user_id, time_slot, mode_per_user_per_interval_JPL_interpolate, most_corr_users_hourly_JPL, similarities_hourly_JPL)

        # If weighted_pred is 0 (i.e., no data available), use user_prediction as a fallback
        user_prediction = mode_per_user_per_interval_JPL_interpolate[user_id][time_slot]
        y_pred_by_user[user_id].append(weighted_pred if weighted_pred != 0 else user_prediction)

# Calculate SMAPE for each user
smape_values = []
for user_id in y_true_by_user:
    y_true = np.array(y_true_by_user[user_id], dtype=float)
    print(len(y_true))
    y_pred = np.array(y_pred_by_user[user_id], dtype=float)
    print(len(y_pred))
    user_smape = calculate_smape(y_true, y_pred)
    if not np.isnan(user_smape):  # Exclude users without predictions
        smape_values.append(user_smape)
        print(f'User {user_id} SMAPE: {user_smape:.2f}%')

# Calculate and print the average SMAPE across users
if smape_values:
    average_smape = np.mean(smape_values)
    print(f'Average SMAPE across users: {average_smape:.2f}%')
else:
    print('No SMAPE values calculated.')


In [ ]:
# Given array
array = JPL_test


# Function to convert hours to HH:MM format, rounding down to the nearest 15 minutes
def hours_to_hhmm_rounded_down(hours):
    total_minutes = hours * 60
    rounded_minutes = total_minutes - (total_minutes % 15)  # Round down to the nearest 15 minutes
    hours_part = int(rounded_minutes // 60)
    minutes_part = int(rounded_minutes % 60)
    return f"{hours_part:02d}:{minutes_part:02d}"

# Apply the function to the first column of the array and create a new column with the results
hhmm_column = np.array([hours_to_hhmm_rounded_down(hours) for hours in array[:, 0]])

array_str = array.astype(str)
hhmm_column = hhmm_column.reshape(-1, 1)  # Reshape for concatenation

# Concatenate the original array with the new HH:MM column
modified_JPL_test= np.concatenate((array_str, hhmm_column), axis=1)

modified_JPL_test


In [ ]:
def ensemble_algorithm_with_correlation(train_data, test_data, test_data_mode, user_id, most_corr_users, dkde_user_model, mode_per_user_per_interval_JPL_interpolate, similarity_matrix, R_SD):
    # Filter training and testing data for the specific user
    user_test_data = test_data[test_data[:, 3] == user_id]
    user_test_data_mode = test_data_mode[test_data_mode[:, 2] == user_id]

    time_slots = list(mode_per_user_per_interval_JPL_interpolate[user_id].keys())
    X_test_dkde_full=[]
    for row in user_test_data:
      if row[-1] in time_slots:
        X_test_dkde_full.append(row)
    #print(X_test_dkde_full)
    # X_test_dkde=[]
    # for row in test_data_mode:
    #   time_slot = row[0]  # Assuming time_slot is at index 0
    #   if time_slot in mode_per_user_per_interval_JPL_interpolate.get(user_id, {}):
    #     if row[2]==user_id:
    #       X_test_dkde.append(row[0])  # true_value is assumed to be at index 3
    X_test_dkde = np.array([row[0] for row in X_test_dkde_full],dtype='float64')
    #X_test_dkde=X_test_dkde_full[:, 0]
    X_test_mlr=user_test_data[:, :2]
    y_test = user_test_data[:, 4]
    y_test_1=user_test_data_mode[:,3]
    y_pred_list = []  # Initialize as a list
    # Choose and use the model based on the R_SD ratio for the main user
    if R_SD.get(user_id, 0) <2:
        user_model = dkde_user_model[user_id]
        y_pred = user_model.predict(X_test_dkde)
        y_pred = [x * 60 for x in y_pred]
        y_pred_list.extend(y_pred)  # Add predictions to the list
    else:
        for row in user_test_data_mode:
            time_slot = row[0]
            if time_slot in mode_per_user_per_interval_JPL_interpolate.get(user_id, {}):
                user_prediction = mode_per_user_per_interval_JPL_interpolate[user_id][time_slot]
                y_pred_list.append(user_prediction)  # Add mode prediction to the list
    y_pred = np.array(y_pred_list)  # Convert list to NumPy array
    #print(len(y_pred))
    # Adjust the prediction based on the most correlated users
    total_correlation = 1
    if user_id in most_corr_users:
        for other_user_id in most_corr_users[user_id]:
            other_user_pred_list = []  # Initialize as a list

            if other_user_id in dkde_user_model:
                correlation = similarity_matrix.loc[user_id, other_user_id]
                if R_SD.get(other_user_id, 0) < 2:
                    other_user_model = dkde_user_model[other_user_id]
                    other_user_pred = other_user_model.predict(X_test_dkde)
                    other_user_pred = [x * 60 for x in other_user_pred]
                    other_user_pred_list.extend(other_user_pred)
                else:
                    for row in user_test_data_mode:
                        time_slot = row[0]
                        if time_slot in mode_per_user_per_interval_JPL_interpolate.get(other_user_id, {}):
                            other_user_prediction = mode_per_user_per_interval_JPL_interpolate[other_user_id][time_slot]
                            other_user_pred_list.append(other_user_prediction)
                other_user_pred = np.array(other_user_pred_list)  # Convert list to NumPy array
                y_pred += other_user_pred * correlation
                total_correlation += correlation

    if total_correlation != 0:
        y_pred /= total_correlation

    # Calculate user SMAPE
    #print("Pred size",len(y_pred))
    y_true=[]
    for row in test_data_mode:
      time_slot = row[0]  # Assuming time_slot is at index 0
      if time_slot in mode_per_user_per_interval_JPL_interpolate.get(user_id, {}):
        if row[2]==user_id:
          y_true.append(row[3])  # true_value is assumed to be at index 3
    #print("True Size",len(y_true))
    n = len(y_true)
    print(len(y_true))
    print(len(y_pred))
    # if R_SD.get(user_id, 0) <3:
    #   smape_val = (1 / n) * np.sum(np.abs(y_test- y_pred) / (np.abs(y_test) + np.abs(y_pred))) * 100
    # else:
    #   smape_val = (1 / n) * np.sum(np.abs(y_test_1- y_pred) / (np.abs(y_test_1 + y_pred))) * 100
    smape_val = (1 / n) * np.sum(np.abs(y_true - y_pred) / (np.abs(y_true+y_pred))) * 100
    return smape_val,total_correlation,y_pred,y_true


In [ ]:
user_test_data = sorted_data_JPL_testing[sorted_data_JPL_testing[:, 2] == 365]

time_slots = list(mode_per_user_per_interval_JPL_interpolate[365].keys())
X_test_dkde_full=[]
for row in user_test_data:
  if row[0] in time_slots:
    X_test_dkde_full.append(row)

In [ ]:
len(X_test_dkde_full)

In [ ]:
converted_array = np.array([
    [float(value) if index < len(row) - 1 else value
     for index, value in enumerate(row)] for row in modified_JPL_test
], dtype=object)

In [ ]:
user_ids_JPL = np.unique(np.concatenate((JPL_train[:, 3], JPL_test[:, 3])))

smape_list_JPL=[]
total_corr=[]
#predicted_values=[]
# true_values=[]
results={}
for user_id in user_ids_JPL:
    smape,corr,predicted_value,true_value=ensemble_algorithm_with_correlation(JPL_train, converted_array, sorted_data_JPL_testing, user_id, most_corr_users_hourly_JPL,DKDE_user_models,mode_per_user_per_interval_JPL_interpolate,similarities_hourly_JPL,R_SD)
    print(f'User {user_id} SMAPE: {smape:.2f}%')
    # predicted_values.append(predicted_value)
    # true_values.append(true_value)
    # smape_list_JPL.append(smape)
    # total_corr.append(corr)
    # Storing the true value and predicted value as a tuple in the dictionary with user_id as the key
    results[user_id] = predicted_value

    smape_list_JPL.append(smape)
    total_corr.append(corr)
# Calculate the average SMAPE
average_smape = np.mean(smape_list_JPL) if smape_list_JPL else 0

print(f"Average SMAPE for the dataset: {average_smape:.2f}%")

In [ ]:
converted_array[0]

In [ ]:
print(len(converted_array[converted_array[:, 3] == 531.0]))

In [ ]:
user_id=405
energy=5.822

user_test_data = sorted_data_JPL_testing[sorted_data_JPL_testing[:, 2] == user_id]

time_slots = list(mode_per_user_per_interval_JPL_interpolate[user_id].keys())
X_test_dkde_full=[]
for row in user_test_data:
  if row[0] in time_slots:
    X_test_dkde_full.append(row)
for index, array in enumerate(X_test_dkde_full):
    if energy in array:
        print(f"The value 7.795 is found in the array at index {index}.")
        break
else:
    print("The value 7.795 was not found in any array.")

value_at_index = results[user_id][index]

print(f"The value at index {index} is {value_at_index}.")
intervals=value_at_index//5+1
intervals

In [ ]:
X_test_dkde_full

In [ ]:
for index, array in enumerate(X_test_dkde_full):
    if energy in array:
        print(index)
        break
else:
    print("The value 7.795 was not found in any array.")

value_at_index = results[user_id][index]

print(f"The value at index {index} is {value_at_index}.")
intervals=value_at_index//5+1
intervals